In [3]:
#imports
import pandas as pd
import numpy as np
import random
import re
import recordlinkage
import time

# prevent depreciation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [5]:
## load in h2a data
h2a = pd.read_excel("../my_data/h2a_2018.xlsx")

## load in investigations/violations data
# url = "../my_data/whd_whisard.csv"
url = "https://enfxfr.dol.gov/data_catalog/WHD/whd_whisard_20210415.csv.zip"
investigations = pd.read_csv(url, 
                      index_col=None,
                      dtype={7:'string'})

## convert the dates to datetime objects
investigations['findings_start_date'] = pd.to_datetime(investigations['findings_start_date'], errors='coerce')
investigations['findings_end_date'] = pd.to_datetime(investigations['findings_end_date'], errors = "coerce")

In [6]:
print("h2a")
h2a.head()
h2a.columns
print('investigations')
investigations.head()

h2a


,CASE_NO,DECISION_DATE,VISA_CLASS,CASE_RECEIVED_DATE,CASE_STATUS,REQUESTED_START_DATE_OF_NEED,REQUESTED_END_DATE_OF_NEED,PRIMARY_SUB,EMPLOYER_NAME,TRADE_NAME_DBA,...,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,OTHER_WORKSITE_LOCATION,ORGANIZATION_FLAG,SWA_NAME,JOB_IDNUMBER,JOB_START_DATE,JOB_END_DATE
0,H-300-17263-482969,2017-10-02 12:46:58,H-2A,2017-09-20,DETERMINATION ISSUED - WITHDRAWN,2017-11-05,2018-05-31,PRI,"Gulf Harvesting, Inc.",NaN,...,Felda,Hendry,FL,33930,Y,H-2A Labor Contractor or Job Contractor,Florida Department of Economic Opportunity,NaN,NaT,NaT
1,H-300-17258-492669,2017-10-02 13:46:39,H-2A,2017-09-15,DETERMINATION ISSUED - CERTIFICATION,2017-12-01,2018-11-29,PRI,ECOSYSTEM CONCEPTS INC.,NaN,...,Dixon,Solano,CA,95620,Y,Association - Filing as Agent (H-2A Only),California Employment Development Department,15575066,2017-09-26,2018-05-31
2,H-300-17240-814010,2017-10-02 15:40:18,H-2A,2017-08-28,DETERMINATION ISSUED - CERTIFICATION EXPIRED,2017-11-01,2018-07-31,PRI,FISHER BROTHERS CATTLE,NaN,...,Anderson,Grimes,TX,77830,Y,Individual Employer,Texas Workforce Commission,TX6572948,2017-08-23,2018-03-17
3,H-300-17244-028701,2017-10-02 15:52:12,H-2A,2017-09-01,DETERMINATION ISSUED - CERTIFICATION EXPIRED,2017-11-01,2018-06-15,PRI,"CIRCLE H CITRUS, INC.",NaN,...,Arcadia,DeSoto,FL,34266,Y,H-2A Labor Contractor or Job Contractor,Department of Economic Opportunity,FL10543312,2017-08-25,2018-02-22
4,H-300-17248-540912,2017-10-02 15:54:55,H-2A,2017-09-05,DETERMINATION ISSUED - PARTIAL CERTIFICATION E...,2017-10-20,2018-03-17,PRI,"FRESH HARVEST, INC.",NaN,...,Mecca,Riverside,CA,92274,Y,H-2A Labor Contractor or Job Contractor,California EDD,15530682,2017-08-31,2018-01-02


Index(['CASE_NO', 'DECISION_DATE', 'VISA_CLASS', 'CASE_RECEIVED_DATE',
       'CASE_STATUS', 'REQUESTED_START_DATE_OF_NEED',
       'REQUESTED_END_DATE_OF_NEED', 'PRIMARY_SUB', 'EMPLOYER_NAME',
       'TRADE_NAME_DBA', 'EMPLOYER_ADDRESS1', 'EMPLOYER_ADDRESS2',
       'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE',
       'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE',
       'EMPLOYER_PHONE_EXT', 'AGENT_POC_EMPLOYER_REP_BY_AGENT', 'LAWFIRM_NAME',
       'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE',
       'JOB_TITLE', 'SOC_CODE', 'SOC_TITLE', 'PRIMARY_CROP', 'NAICS_CODE',
       'NBR_WORKERS_REQUESTED', 'NBR_WORKERS_CERTIFIED', 'FULL_TIME',
       'NATURE_OF_TEMPORARY_NEED', 'BASIC_NUMBER_OF_HOURS',
       'HOURLY_WORK_SCHEDULE_AM', 'HOURLY_WORK_SCHEDULE_PM',
       'BASIC_RATE_OF_PAY', 'OVERTIME_RATE_FROM', 'OVERTIME_RATE_TO',
       'BASIC_UNIT_OF_PAY', 'SUPERVISE_OTHER_EMP', 'SUPERVISE_HOW_MANY',
       'EDUCATION_LEVEL', 'OTHER_EDU', 'M

investigations


,case_id,trade_nm,legal_name,street_addr_1_txt,cty_nm,st_cd,zip_cd,naic_cd,naics_code_description,case_violtn_cnt,...,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt,ld_dt
0,1428484,Reliant Energy,"Reliant Energy Retail Services, LLC",1000 Main,Houston,TX,77002.0,09310,State Generation and Distribution of Electric ...,0,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST
1,1784005,Healthcare Services Group at Westwood Center,"Healthcare Services Group, Inc.",298 Main Street,Keene,NH,3431.0,0967,Local Nursing Homes,1,...,0.0,0,0,0,0.0,0,0,0.0,0,2016-06-11 02:22:25 EDT
2,1460300,Central Avenue Bakery,"Central Avenue Bakery, Inc.",679 Central Avenue,Pawtucket,RI,2861.0,311811,Retail Bakeries,4,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST
3,1294821,Gallagher Services,Catholic Cherities,2520 Pot Spring Road,Lutherville Timonium,MD,21093.0,09690,Local Group Homes for Mentally and Physically ...,0,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST
4,1437099,Dragon Dynasty Oriental Mart,NaN,2299 South Ridgewood Avenue,Daytona Beach,FL,32119.0,42449,Other Grocery and Related Products Merchant Wh...,0,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST


In [7]:
## this function will pull out the certification status from a given h2a application
def find_status(one):
    string_version = str(one)                    ## convert to string
    pattern = r'\-\s(.*)$'                       ## define regex pattern
    found = re.findall(pattern, string_version)  ## search for pattern and return what's found
    return found[0]

h2a["status"] = [find_status(one) for one in h2a.CASE_STATUS]   ## put the status in a new column

## filter to applications that have received certification or partial certification
approved_only = h2a.loc[((h2a.status == "CERTIFICATION") | (h2a.status == "PARTIAL CERTIFICATION")),:].copy()

In [8]:
## this function will clean the EMPLOYER_NAME in approved_only (h2a apps) and legal_name in violations (WHD data)
def clean_names(one):
    string_version = str(one)               ## convert to string
    upper_only = string_version.upper()     ## convert to uppercase
    pattern = r"(LLC|CO|INC)\."             ## locate the LLC, CO, or INC that are followed by a period
    replacement = r'\1'                     ## replace the whole pattern with the LLC/CO/INC component 
    res = re.sub(pattern, replacement, upper_only)  ## compute and return the result
    return res

## make new "name" columns for the cleaned versions of the names
approved_only["name"] = [clean_names(one) for one in approved_only.EMPLOYER_NAME]
approved_only_pure = approved_only.copy()
investigations["name"] = [clean_names(one) for one in investigations.legal_name]
investigations_cleaned = investigations.loc[investigations.name != "NAN",:].copy()      ## get rid of NAN names

In [9]:
investigations_cleaned[["name","h2a_violtn_cnt"]]
# investigations_cleaned.head()
# investigations_cleaned[investigations_cleaned.name.astype(str).str.contains("Jonas", case = False)]

# subset to just those which have violations
violations = investigations_cleaned.loc[investigations_cleaned.h2a_violtn_cnt > 0, :].copy()

,name,h2a_violtn_cnt
0,"RELIANT ENERGY RETAIL SERVICES, LLC",0
1,"HEALTHCARE SERVICES GROUP, INC",0
2,"CENTRAL AVENUE BAKERY, INC",0
3,CATHOLIC CHERITIES,0
5,DERRICK PLUMBING,0
...,...,...
313922,"EL TAPATIO MEXICAN RESTAURANT(LA FINCA, INC)",0
313923,WEST FLEET INC,0
313924,NESBITT FRUIT FARMS,1
313925,"MOTOS, INC",0


In [11]:
## STEP 1: INITIALIZE OUR RECORDLINKAGE INDEX AND BLOCK ON STATE
link_apps_investigations = recordlinkage.Index()                                 
link_apps_investigations.block(left_on = "EMPLOYER_STATE", right_on = "st_cd")     
candidate_links_state = link_apps_investigations.index(approved_only, violations)

## STEP 2: INITIALIZE OUR COMPARE CLASS
compare = recordlinkage.Compare()        
compare.string("name","name", method = "jarowinkler", threshold = 0.85) 

## STEP 3: COMPUTE BASED ON OUR PROVIDED COMPARISON
compare_vectors = compare.compute(candidate_links_state, approved_only, violations)
compare_vectors.columns = ["Name"]                           ## rename the column to appropriate variable
selected = compare_vectors[compare_vectors.Name == 1].copy() ## select the matches

## STEP 4: PULL OUT THE INDEX VALUES SO THAT WE CAN MATCH THE CORRECT ENTRIES TO THE ORIGINAL DATA
n = selected.shape[0]
index_approved_only_values = []
index_violations_cleaned_values = []
for i in range(n):
    index = selected.index[i]
    index_approved_only_values.append(index[0])
    index_violations_cleaned_values.append(index[1])
selected["index_approved_only"] = index_approved_only_values.copy()
selected["index_violations_cleaned"] = index_violations_cleaned_values.copy()

## STEP 5: MERGE THE MATCHES BACK WITH THE APPROVED_ONLY H2A DATA
approved_only["index_approved_only"] = approved_only.index 
approved_only_columns = ["status","JOB_START_DATE","JOB_END_DATE","EMPLOYER_STATE", "name","index_approved_only"] 
m1 = pd.merge(selected, approved_only, on = "index_approved_only", how = "inner")

## STEP 6: MERGE THE RESULT FROM STEP 5 WITH THE VIOLATIONS DATA
violations["index_violations_cleaned"] = violations.index
violations_cleaned_columns = ["st_cd", "name", "index_violations_cleaned", "h2a_violtn_cnt","findings_start_date","findings_end_date"]
m2 = pd.merge(m1, violations[violations_cleaned_columns], on = "index_violations_cleaned", how = "inner", suffixes = ["H2A", "WHD"])

<Index>

<Compare>

In [ ]:
def fuzzyMatch(dbase1, dbase2, blockLeft, blockRight, matchVar1, matchVar2, distFunction, threshold, colsLeft, colsRight):
    link_jobs_debar = recordlinkage.Index() ## initialize our Index
    link_jobs_debar.block(left_on = blockLeft, right_on = blockRight)         ## block on the given block variable

    ## form our index with the two given databases
    candidate_links = link_jobs_debar.index(dbase1, dbase2) 

    compare = recordlinkage.Compare()       ## initialize our compare class
    if (len(matchVar1) != len(matchVar2)):  ## ensure matching num. of matching vars
        print("Need to pass in your matching variables in an array and you need to have the same number of matching variables. Please try again. ")
        return

    for i in range(len(matchVar1)):         ## for each matching pair, add to our comparator
        compare.string(matchVar1[i], matchVar2[i], method = distFunction, threshold = threshold)

    compare_vectors = compare.compute(candidate_links, dbase1, dbase2) ## compute
    compare_vectors 

    # rename columns
    temp_array = []
    for i in range(len(matchVar1)):
        colName = str(matchVar1[i])
        temp_array.append(colName)
    compare_vectors.columns = temp_array

    ## Find the correct selection 
    conditions = []
    for one in matchVar1:
        condition_string = "({one_input} == 1)".format(one_input = one)
        conditions.append(condition_string)
    if (len(conditions) > 1):
        comparison = "&".join(conditions)
    else:
        comparison = conditions[0]
    selected = compare_vectors.query(comparison).copy()

    # Extract index from selection
    n = selected.shape[0]
    index_dbase1_values = []
    index_dbase2_values = []
    for i in range(n):
        index = selected.index[i]
        index_dbase1_values.append(index[0])
        index_dbase2_values.append(index[1])
    selected["index_dbase1"] = index_dbase1_values.copy()
    selected["index_dbase2"] = index_dbase2_values.copy()

    # merge jobs with original columns
    ## this will throw an error if jobs is not the left
    dbase1["index_dbase1"] = dbase1.index
    dbase1_columns = colsLeft
    m1 = pd.merge(selected, dbase1[dbase1_columns], on = "index_dbase1", how = "inner")

    # merge debar with original columns
    dbase2["index_dbase2"] = dbase2.index
    dbase2_columns = colsRight
    m2 = pd.merge(m1, dbase2[dbase2_columns], on = "index_dbase2", how = "inner", suffixes = ["_left", "_right"])

    return m2
    
#################################################################################################
approved_only["city"] = [one.upper() for one in approved_only.EMPLOYER_CITY]
violations["city"] = [one.upper() for one in violations.cty_nm]

#################################################################################################
blockLeft = "EMPLOYER_STATE"
blockRight = "st_cd"
matchingVarsLeft = ["name","city"]
matchingVarsRight = ["name","city"]
colsLeft = ["status","JOB_START_DATE","JOB_END_DATE","EMPLOYER_STATE", "name","index_dbase1","city"]
colsRight = ["st_cd", "name", "h2a_violtn_cnt","findings_start_date","findings_end_date","index_dbase2","city"]

res = fuzzyMatch(approved_only, violations, blockLeft,blockRight,matchingVarsLeft,matchingVarsRight,"jarowinkler",0.85,colsLeft,colsRight)

fuzzy_match_violations = res.loc[(res.h2a_violtn_cnt > 0) & ((res.findings_start_date >= res.JOB_START_DATE)),:].copy()
fuzzy_match_violations

## Above is essentially David's code


In [8]:
# m2.columns
# m2.head()
print('we found %s unique employers in the 2018 H2A with violations' %m2.nameH2A.nunique())

we found 1905 unique employers in the 2018 H2A with violations


In [13]:
# Make a classifier for Y if the name in the H2A was fuzzy matched in m2
approved_only_pure["is_violator"] = np.where(approved_only_pure.name.isin(list(fuzzy_match_violations.name_y)), 1, 0)
approved_only_pure.is_violator.value_counts()
#approved_only_pure.head()

0    8126
1    3901
Name: is_violator, dtype: int64

In [14]:
approved_only_pure.dtypes

CASE_NO                       object
DECISION_DATE         datetime64[ns]
VISA_CLASS                    object
CASE_RECEIVED_DATE    datetime64[ns]
CASE_STATUS                   object
                           ...      
JOB_START_DATE        datetime64[ns]
JOB_END_DATE          datetime64[ns]
status                        object
name                          object
is_violator                    int64
Length: 65, dtype: object

In [55]:
print("there are %s applications in the H2A approved Dataset" %len(approved_only_pure))
print('but only %s unique companies within those applications' %approved_only_pure.name.nunique())

def find_pattern(string, pat):
    res = re.findall(pat, string)
    if (len(res) > 0):
        return True
    else: 
        return False

def find_mode(col_of_interest):
    list_version = list(col_of_interest.values)
    values = sorted(list_version, key = list_version.count, reverse=True)  ## sorted the adj_only list in descending order
    values_no_dups = list(dict.fromkeys(values))                      ## remove duplicates while preserving order to get top 5
    return values_no_dups[0]  


def form_representative(df, col_to_groupby):
    list_of_reps = []
    for one in df[col_to_groupby].unique():
        t = time.time()
        temp_df = df.loc[df[col_to_groupby] == one].copy()
        to_add = {}
        for col in temp_df:
            col_type = df.dtypes[col]
            if (col_type == "int64"):
                to_add[col] = temp_df[col].mean(skipna = True)
            elif (col_type == "object"):
                to_add[col] = find_mode(temp_df[col])
            elif (col_type == "float64"):
                to_add[col] = temp_df[col].mean(skipna = True)
            elif (col_type == "datetime64[ns]"):
                if (find_pattern(str(col),r'START')):
                    to_add[col] = temp_df[col].min()
                elif (find_pattern(str(col),r'END')):
                    to_add[col] = temp_df[col].max()
                else:
                    to_add[col] = temp_df[col].min()
            else:
                print("Other type")
        list_of_reps.append(to_add)
        elapsed = time.time() - t

        print("done with one, took " + str(elapsed) + " seconds")
    
    res = pd.DataFrame(list_of_reps)
    return res

test_res = form_representative(approved_only_pure, "name")
test_res.head()
        
# approved_only_pure = approved_only_pure.drop_duplicates(subset=['name'])
# approved_only_pure.is_violator.value_counts()

0.03947091102600098 seconds
done with one, took 0.03352689743041992 seconds
done with one, took 0.04239797592163086 seconds
done with one, took 0.08683204650878906 seconds
done with one, took 0.05241894721984863 seconds
done with one, took 0.043827056884765625 seconds
done with one, took 0.025059938430786133 seconds
done with one, took 0.028671979904174805 seconds
done with one, took 0.036492109298706055 seconds
done with one, took 0.10295605659484863 seconds
done with one, took 0.07209205627441406 seconds
done with one, took 0.03780484199523926 seconds
done with one, took 0.03844571113586426 seconds
done with one, took 0.028826236724853516 seconds
done with one, took 0.034240007400512695 seconds
done with one, took 0.062463998794555664 seconds
done with one, took 0.03725004196166992 seconds
done with one, took 0.061985015869140625 seconds
done with one, took 0.03646087646484375 seconds
done with one, took 0.03721117973327637 seconds
done with one, took 0.02457904815673828 seconds
done

,CASE_NO,DECISION_DATE,VISA_CLASS,CASE_RECEIVED_DATE,CASE_STATUS,REQUESTED_START_DATE_OF_NEED,REQUESTED_END_DATE_OF_NEED,PRIMARY_SUB,EMPLOYER_NAME,TRADE_NAME_DBA,...,WORKSITE_POSTAL_CODE,OTHER_WORKSITE_LOCATION,ORGANIZATION_FLAG,SWA_NAME,JOB_IDNUMBER,JOB_START_DATE,JOB_END_DATE,status,name,is_violator
0,H-300-17258-492669,2017-10-02 13:46:39,H-2A,2017-09-15,DETERMINATION ISSUED - CERTIFICATION,2017-12-01,2018-11-29,PRI,ECOSYSTEM CONCEPTS INC.,NaN,...,95620,Y,Association - Filing as Agent (H-2A Only),California Employment Development Department,15575066,2017-09-26,2018-05-31,CERTIFICATION,ECOSYSTEM CONCEPTS INC,0.0
1,H-300-17257-446860,2017-10-02 16:36:19,H-2A,2017-08-17,DETERMINATION ISSUED - CERTIFICATION,2017-10-16,2019-12-06,PRI,WESTERN RANGE ASSOCIATION,NaN,...,93635,Y,Association - Joint Employer (H-2A Only),California Employment Development Department,NaN,2017-08-22,2019-09-07,CERTIFICATION,WESTERN RANGE ASSOCIATION,1.0
2,H-300-17257-446860,2017-10-02 16:36:19,H-2A,2017-09-14,DETERMINATION ISSUED - CERTIFICATION,2017-12-08,2019-09-06,SUB,SHEEP COMPANY LLC,NaN,...,93210,Y,NaN,California Employment Development Department,15567271,2017-09-20,2019-03-08,CERTIFICATION,SHEEP COMPANY LLC,0.0
3,H-300-17262-816860,2017-10-02 16:44:54,H-2A,2017-09-19,DETERMINATION ISSUED - CERTIFICATION,2018-01-01,2018-12-30,PRI,Sieben Ranch Co.,NaN,...,59648,Y,Association - Filing as Agent (H-2A Only),Montana State Workforce,10316116,2017-09-21,2018-06-29,CERTIFICATION,SIEBEN RANCH CO,0.0
4,H-300-17256-339375,2017-10-03 13:28:16,H-2A,2017-09-13,DETERMINATION ISSUED - CERTIFICATION,2017-12-08,2019-11-06,SUB,TUTTLE LIVESTOCK COMPANY LLC,NaN,...,81625,Y,NaN,Craig Workforce Center,CO 7136889,2017-09-15,2019-05-08,CERTIFICATION,TUTTLE LIVESTOCK COMPANY LLC,1.0


In [57]:
test_res.to_csv("repMatrix.csv")
approved_only_pure.to_csv('preMatrix.csv')

In [58]:
test_res.shape

(7643, 65)